In [0]:
%pip install confluent_kafka
%pip install Faker

Python interpreter will be restarted.
Requirement already satisfied: confluent_kafka in /local_disk0/.ephemeral_nfs/envs/pythonEnv-341b20a4-7e83-4002-91e2-8392da2568db/lib/python3.8/site-packages (2.4.0)
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: Faker in /local_disk0/.ephemeral_nfs/envs/pythonEnv-341b20a4-7e83-4002-91e2-8392da2568db/lib/python3.8/site-packages (26.0.0)
Requirement already satisfied: python-dateutil>=2.4 in /databricks/python3/lib/python3.8/site-packages (from Faker) (2.8.1)
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.8/site-packages (from python-dateutil>=2.4->Faker) (1.15.0)
Python interpreter will be restarted.

In [0]:
from confluent_kafka import Producer
from faker import Faker
import time
import random

In [0]:
UsuarioKafka = dbutils.secrets.get(scope='sc-adls', key='UsuarioKafka')
PasswordKafka = dbutils.secrets.get(scope='sc-adls', key='PasswordKafka')
ServerKafka = dbutils.secrets.get(scope='sc-adls', key='ServerKafka')

In [0]:
# Configuración del productor de Kafka

conf = {
    'bootstrap.servers': ServerKafka,
    'security.protocol': 'SASL_SSL',
    'sasl.mechanisms': 'PLAIN',
    'sasl.username': UsuarioKafka,
    'sasl.password': PasswordKafka,
}

producer = Producer(conf) # Envio de mensajes al topico de kafka
fake = Faker() # Diccionario para conectar y autentificar el productor con el cluster
topic = 'notasAlumnos'

def mensajes_entrega(msg_error, msg):
    if msg_error is not None:
        print('Mensaje de error: {}'.format(msg_error))
    #else:
    #    print('Mensaje entregado a {} Partición[{}]'.format(msg.topic(), msg.partition()))


In [0]:
def generar_datos(num_registros):
    # Definir cursos posibles y una función para generar datos
    cursos = ['apache_spark', 'kafka', 'databricks', 'Data Factory', 'Hadoop']

    datos = []
    for _ in range(num_registros):
        # Generar un DNI de 4 caracteres
        dni = ''.join([str(random.randint(0, 4)) for _ in range(4)])
        fullName = fake.name()
        curso = random.choice(cursos)  # Seleccionar un curso al azar
        nota = random.randint(7, 20)
        timestamp = fake.date_time_this_decade()
        datos.append({
            'dni': dni,
            'nombres': fullName,
            'curso': curso,
            'nota': nota,
            'fechaRegistro': timestamp.strftime('%m/%d/%Y %H:%M')
        })

        # Envía un mensaje al tópico
        producer.produce(topic, key=fake.uuid4(), value=str(datos[-1]), callback=mensajes_entrega)
        producer.poll(0)  # Verificar cualquier evento pendiente con la entrega de mensajes
        time.sleep(1)  # Envía un mensaje por segundo

        producer.flush()  # Enviar todos los mensajes al productor
    return datos

# Generar datos
datos_generados = generar_datos(10)